In [6]:
import random
import time
from z3 import Optimize, Real, Int, If, And, ToInt, sat

def estate_tax_z3_solver(
    # 新增 free_vars 參數
    free_vars: list = None,
    # 使用者輸入：死亡日期區間、是否為軍警公教因公死亡
    death_period: int = 1,
    is_military_police: bool = False,
    
    # 遺產總額相關
    land_value: float = 0.0,           
    building_value: float = 0.0,       
    house_value: float = 0.0,          
    deposit_bonds_value: float = 0.0,  
    stock_invest_value: float = 0.0,   
    cash_gold_jewelry_value: float = 0.0,
    gift_in_2yrs_value: float = 0.0,   
    
    # 扣除額（人數或金額）
    spouse_count: int = 0,            
    lineal_descendant_count: int = 0,  
    father_mother_count: int = 0,      
    disability_count: int = 0,         
    dependent_count: int = 0,          

    farmland_val: float = 0.0,           
    inheritance_6to9_val: float = 0.0,   
    unpaid_tax_fines_val: float = 0.0,   
    unpaid_debts_val: float = 0.0,       
    will_management_fee: float = 0.0,    
    public_facility_retention_val: float = 0.0,
    spouse_surplus_right_val: float = 0.0,
    
    # 扣抵稅額
    gift_tax_offset: float = 0.0,    
    foreign_tax_offset: float = 0.0  
):
    # -------------------------------------------------------
    # 1. 計算 base_exemption_val 與 funeral_val
    # -------------------------------------------------------
    if death_period in [1, 2]:
        base_exemption_val = 12_000_000
        funeral_val = 1_230_000
    else:
        base_exemption_val = 13_330_000
        funeral_val = 1_230_000 if death_period in [1,2,3] else 1_380_000
    if is_military_police:
        base_exemption_val *= 2

    # -------------------------------------------------------
    # 2. 建立 Optimize 以及所有 Z3 變數
    # -------------------------------------------------------
    opt = Optimize()

    # 資產項目
    land_z    = Real('land_z');    building_z = Real('building_z')
    house_z   = Real('house_z');   deposit_z  = Real('deposit_z')
    stock_z   = Real('stock_z');   cash_z     = Real('cash_z')
    gift2_z   = Real('gift2_z')

    # 扣除額項目
    spouse_ct_z   = Real('spouse_ct_z')
    lineal_ct_z   = Real('lineal_ct_z')
    fm_ct_z       = Real('fm_ct_z')
    disability_z  = Real('disability_z')
    dependent_z   = Real('dependent_z')

    farmland_z    = Real('farmland_z')
    inh6to9_z     = Real('inh6to9_z')
    fines_z       = Real('fines_z')
    debts_z       = Real('debts_z')
    willfee_z     = Real('willfee_z')
    publicfac_z   = Real('publicfac_z')
    spousesur_z   = Real('spousesur_z')

    # 扣抵稅額
    giftoffset_z  = Real('giftoffset_z')
    foreigntax_z  = Real('foreigntax_z')

    # 中間計算
    inheritance_total_z    = Real('inheritance_total_z')
    base_exemption_z      = Real('base_exemption_z')
    total_deduction_z     = Real('total_deduction_z')
    taxable_inheritance_z = Real('taxable_inheritance_z')
    bracket_rate_z        = Real('bracket_rate_z')
    bracket_diff_z        = Real('bracket_diff_z')
    offsets_z             = Real('offsets_z')
    estate_tax_z          = Real('estate_tax_z')
    
    # 將 Real 的 estate_tax_z 取整給一個 Int
    final_tax = Int('final_tax')
    opt.add(final_tax == ToInt(estate_tax_z))

    # 淨課稅遺產淨額回傳用
    net_taxable_income_z = taxable_inheritance_z

    # -------------------------------------------------------
    # 3. 建立 params dict，並依 free_vars 決定要不要加上「== 原值」
    # -------------------------------------------------------
    if free_vars is None:
        free_vars = []
    nonneg = lambda v: v >= 0
    params = {
        'land_value':    (land_z,    land_value,    [nonneg]),
        'building_value':(building_z,building_value,[nonneg]),
        'house_value':   (house_z,   house_value,   [nonneg]),
        'deposit_bonds_value':(deposit_z, deposit_bonds_value, [nonneg]),
        'stock_invest_value': (stock_z,   stock_invest_value,  [nonneg]),
        'cash_gold_jewelry_value': (cash_z, cash_gold_jewelry_value, [nonneg]),
        'gift_in_2yrs_value': (gift2_z, gift_in_2yrs_value, [nonneg]),

        'spouse_count':  (spouse_ct_z, spouse_count,  [nonneg]),
        'lineal_descendant_count': (lineal_ct_z, lineal_descendant_count, [nonneg]),
        'father_mother_count':     (fm_ct_z,     father_mother_count,     [nonneg]),
        'disability_count':        (disability_z, disability_count,        [nonneg]),
        'dependent_count':         (dependent_z,  dependent_count,         [nonneg]),

        'farmland_val':            (farmland_z,    farmland_val,            [nonneg]),
        'inheritance_6to9_val':    (inh6to9_z,     inheritance_6to9_val,    [nonneg]),
        'unpaid_tax_fines_val':    (fines_z,       unpaid_tax_fines_val,    [nonneg]),
        'unpaid_debts_val':        (debts_z,       unpaid_debts_val,        [nonneg]),
        'will_management_fee':     (willfee_z,     will_management_fee,     [nonneg]),
        'public_facility_retention_val': (publicfac_z, public_facility_retention_val, [nonneg]),
        'spouse_surplus_right_val': (spousesur_z, spouse_surplus_right_val, [nonneg]),

        'gift_tax_offset':         (giftoffset_z,  gift_tax_offset,         [nonneg]),
        'foreign_tax_offset':      (foreigntax_z,  foreign_tax_offset,      [nonneg]),
    }

    for name, (z3var, orig, cons_list) in params.items():
        for c in cons_list:
            opt.add(c(z3var))
        if name not in free_vars:
            opt.add(z3var == orig)

    # -------------------------------------------------------
    # 4. 加入其餘約束
    # -------------------------------------------------------
    opt.add(inheritance_total_z ==
            land_z + building_z + house_z +
            deposit_z + stock_z + cash_z + gift2_z)
    opt.add(base_exemption_z == base_exemption_val)

    # 各項扣除
    spouse_deduction_z = Real('spouse_deduction_z')
    lineal_deduction_z = Real('lineal_deduction_z')
    fm_deduction_z     = Real('fm_deduction_z')
    dis_deduction_z    = Real('dis_deduction_z')
    dep_deduction_z    = Real('dep_deduction_z')
    funeral_deduction_z= Real('funeral_deduction_z')

    opt.add(spouse_deduction_z ==
            spouse_ct_z * If(death_period in (4,5), 5_530_000, 4_930_000))
    opt.add(lineal_deduction_z ==
            lineal_ct_z * If(death_period in (4,5), 560_000, 500_000))
    opt.add(fm_deduction_z ==
            fm_ct_z * If(death_period in (4,5), 1_380_000, 1_230_000))
    opt.add(dis_deduction_z ==
            disability_z * If(death_period in (4,5), 6_930_000, 6_180_000))
    opt.add(dep_deduction_z ==
            dependent_z * If(death_period in (4,5), 560_000, 500_000))
    opt.add(funeral_deduction_z == funeral_val)

    opt.add(total_deduction_z ==
            spouse_deduction_z + lineal_deduction_z +
            fm_deduction_z + dis_deduction_z + dep_deduction_z +
            farmland_z + inh6to9_z + fines_z + debts_z +
            funeral_deduction_z + willfee_z + publicfac_z + spousesur_z)

    opt.add(taxable_inheritance_z ==
            inheritance_total_z - base_exemption_z - total_deduction_z)

    # 稅率級距
    opt.add(
        If(death_period == 1,
           And(bracket_rate_z == 0.10, bracket_diff_z == 0),
        If(death_period == 5,
           If(taxable_inheritance_z <= 56_210_000,
              And(bracket_rate_z == 0.10, bracket_diff_z == 0),
           If(taxable_inheritance_z <= 112_420_000,
              And(bracket_rate_z == 0.15, bracket_diff_z == 2_810_500),
              And(bracket_rate_z == 0.20, bracket_diff_z == 8_431_500))),
        If(taxable_inheritance_z <= 50_000_000,
           And(bracket_rate_z == 0.10, bracket_diff_z == 0),
        If(taxable_inheritance_z <= 100_000_000,
           And(bracket_rate_z == 0.15, bracket_diff_z == 2_500_000),
           And(bracket_rate_z == 0.20, bracket_diff_z == 7_500_000)))))  
    )

    opt.add(offsets_z == giftoffset_z + foreigntax_z)

    opt.add(
        estate_tax_z ==
        If(taxable_inheritance_z <= 0, 0,
           If(taxable_inheritance_z * bracket_rate_z - bracket_diff_z - offsets_z < 0,
              0,
              taxable_inheritance_z * bracket_rate_z - bracket_diff_z - offsets_z))
    )

    # -------------------------------------------------------
    # 5. 初始計算（push/pop）
    # -------------------------------------------------------
    if free_vars:
        opt.push()
        for name in free_vars:
            z3var, orig, _ = params[name]
            opt.add(z3var == orig)
        if opt.check() == sat:
            fixed_tax = opt.model()[final_tax].as_long()
        else:
            fixed_tax = None
        opt.pop()
    else:
        fixed_tax = opt.model()[final_tax].as_long() if opt.check() == sat else None

    print(f"你的稅務運算結果為：{fixed_tax}（固定所有參數時的初始值）")
    print("選擇可調整變數為：", free_vars)
    print("--------------------------------------------------\n")

    # -------------------------------------------------------
    # 6. 最小化迭代
    # -------------------------------------------------------
    best_tax = None
    iteration = 0
    while True:
        if opt.check() != sat:
            print(f"第 {iteration+1} 次最佳化：unsat，無解，已是最佳解")
            break
        m = opt.model()
        current_tax = m[final_tax].as_long()
        if iteration == 0 or current_tax < best_tax:
            best_tax = current_tax
            print(f"----- 第 {iteration+1} 次最佳化 -----")
            print("當前稅額:", current_tax)
            diffs = {}
            for name, (z3var, orig, _) in params.items():
                try:
                    val = m.eval(z3var).as_long()
                except:
                    val = None
                if val is not None and val != orig:
                    diffs[name] = {"original": orig, "optimized": val, "difference": val - orig}
            if diffs:
                print("變更的變數:")
                for k,v in diffs.items():
                    print(f"  {k}: 原={v['original']}, 優={v['optimized']}, 差={v['difference']}")
            print("-------------------------")
        else:
            print(f"第 {iteration+1} 次最佳化：無變化，已是最佳解")
            break

        opt.add(final_tax < best_tax)
        iteration += 1

    print(f"\n=== 最終最佳解：經過 {iteration} 次最佳化，最終稅額 = {best_tax} ===")
    print("--------------------------------------------------\n")

    # -------------------------------------------------------
    # 7. 收集最終參數與差異
    # -------------------------------------------------------
    final_params = {}
    differences = {}
    for name, (z3var, orig, _) in params.items():
        try:
            val = m.eval(z3var).as_long()
        except:
            val = None
        final_params[name] = {"value": val, "type": "free" if name in free_vars else "fixed"}
        if val is not None and val != orig:
            differences[name] = {"original": orig, "optimized": val, "difference": val - orig}

    return best_tax, m[net_taxable_income_z].as_long(), final_params, differences


In [7]:
# 所有參數都當固定值，不做最佳化
res1 = estate_tax_z3_solver(
    death_period=5,
    is_military_police=False,
    land_value=100_000_000,
    building_value=20_000_000,
    house_value=10_000_000,
    deposit_bonds_value=5_000_000,
    stock_invest_value=10_000_000,
    cash_gold_jewelry_value=2_000_000,
    gift_in_2yrs_value=1_000_000,
    spouse_count=1,
    lineal_descendant_count=2,
    father_mother_count=0,
    disability_count=0,
    dependent_count=0,
    farmland_val=0,
    inheritance_6to9_val=0,
    unpaid_tax_fines_val=0,
    unpaid_debts_val=0,
    will_management_fee=0,
    public_facility_retention_val=0,
    spouse_surplus_right_val=0,
    gift_tax_offset=0,
    foreign_tax_offset=0
)
print(res1)


你的稅務運算結果為：16896500（固定所有參數時的初始值）
選擇可調整變數為： []
--------------------------------------------------

----- 第 1 次最佳化 -----
當前稅額: 16896500
-------------------------
第 2 次最佳化：unsat，無解，已是最佳解

=== 最終最佳解：經過 1 次最佳化，最終稅額 = 16896500 ===
--------------------------------------------------

(16896500, 126640000, {'land_value': {'value': 100000000, 'type': 'fixed'}, 'building_value': {'value': 20000000, 'type': 'fixed'}, 'house_value': {'value': 10000000, 'type': 'fixed'}, 'deposit_bonds_value': {'value': 5000000, 'type': 'fixed'}, 'stock_invest_value': {'value': 10000000, 'type': 'fixed'}, 'cash_gold_jewelry_value': {'value': 2000000, 'type': 'fixed'}, 'gift_in_2yrs_value': {'value': 1000000, 'type': 'fixed'}, 'spouse_count': {'value': 1, 'type': 'fixed'}, 'lineal_descendant_count': {'value': 2, 'type': 'fixed'}, 'father_mother_count': {'value': 0, 'type': 'fixed'}, 'disability_count': {'value': 0, 'type': 'fixed'}, 'dependent_count': {'value': 0, 'type': 'fixed'}, 'farmland_val': {'value': 0, 'type':

In [17]:
# 只把 land_value 當做自由變數，讓 Z3 幫我們調整它以求最小化稅額
res2 = estate_tax_z3_solver(
    free_vars=['land_value'],
    death_period=5,
    is_military_police=False,
    land_value=100_000_000,
    building_value=20_000_000,
    house_value=10_000_000,
    deposit_bonds_value=5_000_000,
    stock_invest_value=10_000_000,
    cash_gold_jewelry_value=2_000_000,
    gift_in_2yrs_value=1_000_000,
    spouse_count=1,
    lineal_descendant_count=2,
    father_mother_count=0,
    disability_count=0,
    dependent_count=0,
    farmland_val=0,
    inheritance_6to9_val=0,
    unpaid_tax_fines_val=0,
    unpaid_debts_val=0,
    will_management_fee=0,
    public_facility_retention_val=0,
    spouse_surplus_right_val=0,
    gift_tax_offset=0,
    foreign_tax_offset=0
)
print(res2)


你的稅務運算結果為：16896500（固定所有參數時的初始值）
選擇可調整變數為： ['land_value']
--------------------------------------------------

----- 第 1 次最佳化 -----
當前稅額: 14052500
變更的變數:
  land_value: 原=100000000, 優=85780001, 差=-14219999
-------------------------
----- 第 2 次最佳化 -----
當前稅額: 5621000
變更的變數:
  land_value: 原=100000000, 優=29570000, 差=-70430000
-------------------------
----- 第 3 次最佳化 -----
當前稅額: 5620999
變更的變數:
  land_value: 原=100000000, 優=29569990, 差=-70430010
-------------------------
----- 第 4 次最佳化 -----
當前稅額: 5620998
變更的變數:
  land_value: 原=100000000, 優=29569980, 差=-70430020
-------------------------
----- 第 5 次最佳化 -----
當前稅額: 5620997
變更的變數:
  land_value: 原=100000000, 優=29569970, 差=-70430030
-------------------------
----- 第 6 次最佳化 -----
當前稅額: 5620996
變更的變數:
  land_value: 原=100000000, 優=29569960, 差=-70430040
-------------------------
----- 第 7 次最佳化 -----
當前稅額: 2664000
變更的變數:
  land_value: 原=100000000, 優=0, 差=-100000000
-------------------------
第 8 次最佳化：unsat，無解，已是最佳解

=== 最終最佳解：經過 7 次最佳化，最終稅額 = 2664000 ===

In [19]:
res3 = estate_tax_z3_solver(
    free_vars=['gift_in_2yrs_value','cash_gold_jewelry_value'],
    death_period=5,
    is_military_police=False,
    land_value=100_000_000,
    building_value=20_000_000,
    house_value=10_000_000,
    deposit_bonds_value=5_000_000,
    stock_invest_value=10_000_000,
    cash_gold_jewelry_value=2_000_000,
    gift_in_2yrs_value=1_000_000,
    spouse_count=1,
    lineal_descendant_count=2,
    father_mother_count=0,
    disability_count=0,
    dependent_count=0,
    farmland_val=0,
    inheritance_6to9_val=0,
    unpaid_tax_fines_val=0,
    unpaid_debts_val=0,
    will_management_fee=0,
    public_facility_retention_val=0,
    spouse_surplus_right_val=0,
    gift_tax_offset=0,
    foreign_tax_offset=0
)
print(res3)


你的稅務運算結果為：16896500（固定所有參數時的初始值）
選擇可調整變數為： ['gift_in_2yrs_value', 'cash_gold_jewelry_value']
--------------------------------------------------

----- 第 1 次最佳化 -----
當前稅額: 16296500
變更的變數:
  cash_gold_jewelry_value: 原=2000000, 優=0, 差=-2000000
  gift_in_2yrs_value: 原=1000000, 優=0, 差=-1000000
-------------------------
第 2 次最佳化：unsat，無解，已是最佳解

=== 最終最佳解：經過 1 次最佳化，最終稅額 = 16296500 ===
--------------------------------------------------

(16296500, 123640000, {'land_value': {'value': 100000000, 'type': 'fixed'}, 'building_value': {'value': 20000000, 'type': 'fixed'}, 'house_value': {'value': 10000000, 'type': 'fixed'}, 'deposit_bonds_value': {'value': 5000000, 'type': 'fixed'}, 'stock_invest_value': {'value': 10000000, 'type': 'fixed'}, 'cash_gold_jewelry_value': {'value': 0, 'type': 'free'}, 'gift_in_2yrs_value': {'value': 0, 'type': 'free'}, 'spouse_count': {'value': 1, 'type': 'fixed'}, 'lineal_descendant_count': {'value': 2, 'type': 'fixed'}, 'father_mother_count': {'value': 0, 'type': 'fix